## Result (2023-02)
    1. 레이블별 해상도 체크: 모두 1920x1080이 아님 
    2. 너무 작은 width, height를 가지는 데이터들은 제외됨
    3. 관심영역의 크기가 편차가 심함, 256, 384, 512로 먼저 생성
        - 이후 그래픽카드 가용 메모리에 따라 조정
        - 카메라 구도에 따른 (1)입력 이미지 해상도 축소와 늘리는 방법, (2)제로 패딩 시도 해봐야 함
        - 카메라 해상도가 다를 때 사용하는 기술들의 논문 찾기 필요
        - 이미지 전처리 방법 적용 방안 (논문 참고)
    4. 제외된 이미지들이 있으며, 해당 이미지들은 매우 가까이서 찍은 사진들로 따로 추출이 필요할 수 있음
    5. PCA, TSNE

## processing (2023-03-13)
1. 레이블별 해상도 이슈 (해상도는 모두 다르기야 하겠지만, 카메라의 구도 문제?)
2. 너무 작은 데이터, 너무 큰 데이터의 분포 확인
3. 전처리의 방법
- resize (보간법 4, 8), transforms.resize
- ratio + zero_padding (보간법 4, 8)
- 보간법 cubic으로 진행
4. pca, tsne를 통해 데이터의 모인 정도를 확인

In [1]:
# default
from pprint import pprint
import natsort
import os, sys
import json
import pickle as pkl
from tqdm import notebook

# data structure
import numpy as np
from PIL import Image

# visualize
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# preprocessing
import cv2

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE

In [4]:
TRAIN = 0
VALID = 1
data_type = ["train", "valid"]
data_label = ["A1", "A2", "A3", "A4", "A5", "A6"]

project_path = "D:/Capstone/"

In [20]:
csv_data_list = list()

for label in data_label:
    csv_path = f"{project_path}{label}.csv"
    csv_data = pd.read_csv(csv_path, encoding="CP949")
    
    # preprocessing
    exist_index = csv_data.loc[csv_data["exist"] == "Y"].index
    csv_data = csv_data.loc[exist_index].copy()
    csv_data.drop(["Unnamed: 0", "exist", "diagnosis", "src_path", "label_path", "type", "fileformat", "copyrighter"], axis=1, inplace=True)
    
    csv_data_list.append(csv_data)